# 機械学習 ハンズオン
本資料では、`scikit-learn`を用いた機械学習を実際に体験していただきます。

Google Colaboratoryでは`scikit-learn`がインストール済みですので、すぐにプログラムを作りはじめることができます。

`scikit-learn`がインストールされていない環境では、次のとおり`pip`コマンドなどでインストールを行ってください。

```python
pip install scikit-learn
```

# ファイルのダウンロード
あらかじめ、ハンズオンで必要なファイルをダウンロードしておきます。

In [ ]:
!curl -O https://raw.githubusercontent.com/tendo-sms/python_intermediate/refs/heads/main/2025/data/california_housing.csv

# 事前準備 (乱数seedの固定)
scikit-learnによる機械学習は処理の中で乱数が使われるため、通常は実行するたびに結果が変わります。

しかし今回は講義での解説用に、乱数seedを固定して実行結果が毎回同じになるようにしておきます。

以下のプログラムを最初に実行しておいてください。

In [ ]:
import numpy as np
import random

# この数値を変更すると、本資料中のプログラムの実行結果も少し変わります
SEED = 42

np.random.seed(SEED)
random.seed(SEED)

もし、これ以降のプログラムを実行したときに「変数SEEDが存在しない」というエラーとなった場合は、上記のプログラムが実行されていないことが原因です。忘れずに実行してください。

それでは、機械学習の基本タスクである「回帰」「分類」「クラスタリング」「次元削減」それぞれについて、実際にプログラムを作成して体験してみましょう。

# 回帰
はじめは、回帰を実践します。

今回は、`california housing dataset`(カリフォルニアの住宅価格データセット)を用いて、住宅に関する様々な特徴をもとに、住宅価格の予想を行ってみます。

`california housing dataset`は、「1990年の米国国勢調査から得られたカリフォルニア州の住宅価格」の表形式データセット(=構造化データセット)です。次のような特徴量を持っています。

| カラム名 | 意味 | 説明 |
|-----------|----------------|------|
| MedInc | 世帯の中央値所得 | 地域内の世帯年収の中央値(単位:1万ドル) |
| HouseAge | 住宅の平均築年数 | 地域内の住宅の平均築年数 |
| AveRooms | 世帯あたりの平均部屋数 | 地域内の住宅の部屋数の平均(部屋の総数 ÷ 世帯数) |
| AveBedrms | 世帯あたりの平均寝室数 | 地域内の住宅の寝室数の平均(寝室の総数 ÷ 世帯数) |
| Population | 人口 | 地域内の人口総数 |
| AveOccup | 世帯あたりの平均居住者数 | 住宅1軒あたりの平均居住者数(人口 ÷ 世帯数) |
| Latitude | 緯度 | 地域の緯度(地理座標) |
| Longitude | 経度 | 地域の経度(地理座標) |
| MedHouseVal | 住宅価格の中央値 | 地域内の住宅価格の中央値(単位:10万ドル) |

MedHouseVal(住宅価格の中央値)を予想するのが、今回のタスクです。


## データの準備
`california housing dataset`は、`scikit-learn`で標準提供されていますが、今回は教育用に少しアレンジしたデータを使用します。

あらかじめダウンロードした`california_housing.csv`を、`pandas`のデータフレームとして読み込みます。



In [ ]:
import pandas as pd

# データフレームを改行なしでprintするための設定
pd.set_option('display.expand_frame_repr', False)

data = pd.read_csv("california_housing.csv")
data

## 探索的データ分析(EDA)
EDAの例として、ここでは特徴量間の関係をいくつか可視化してみます。

### 統計量・欠損値等の確認
データフレームのいくつかのメソッドを使って、データの概要を調べてみます。

In [ ]:
print("=== データ形状 ===")
print(data.shape)

print("\n=== データ型 ===")
print(data.dtypes)

print("\n=== 欠損値確認 ===")
print(data.isnull().sum())

print("\n=== 統計量 ===")
print(data.describe())

気になる点として、一定数の欠損値が見られます。

### 特徴量間の関係 (ピアソン相関)
すべての特徴量同士のピアソン相関を示す、ヒートマップ図を作成してみます。

EDAでの可視化には、`matplotlib`をラッピングしてより美しい図を簡単に描けるようにした`seaborn`パッケージが便利です。

`seaborn`をインポートする時は`sns`という別名をつけるのが一般的です。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.show()

値が+1に近い組み合わせほど正の相関が強く、-1に近い組み合わせほど負の相関が強いといえます。

AveRooms(部屋数)とAveBedrms(寝室数)に高い相関がみられますが、これは当然と言えるでしょう。

Latitude(緯度)とLongitude(経度)に高い相関がみられますが、これはカリフォルニア州が、北西から南東に向けて細長く斜めに伸びる地域であることが原因と推定できます。

今回の目的であるMedHouseVal(住宅価格)は、MedInc(世帯所得)との相関が比較的高いようです。


### 特徴量間の関係 (散布図ペアプロット)
いくつかの特徴量同士の散布図を作成してみます。

In [ ]:
sns.pairplot(data[['MedInc', 'HouseAge', 'AveRooms', 'MedHouseVal']])
plt.show()

ヒートマップ図からも確認できたように、MedIncとMedHouseValの相関関係が散布図でも確認できます。

### 特徴量間の関係(相互情報量)
ピアソン相関や散布図は、線形関係の確認のみに着目した手法でした。

ここでは、非線形の相関も確認するため、相互情報量(MI)によるヒートマップ図も作成してみます。

`scikit-learn`の`mutual_info_regression`を使って、確認してみます。

In [ ]:
from sklearn.feature_selection import mutual_info_regression

# mutual_info_regressionはNaNを扱えないため除去
data = data.dropna()

# 空の行列を作成
cols = data.columns
mi_matrix = pd.DataFrame(np.zeros((len(cols), len(cols))), index=cols, columns=cols)

# 全カラム間のMIを計算
for i, col_x in enumerate(cols):
    for j, col_y in enumerate(cols):
        if i == j:
            # 自己相関はNaNを設定
            mi_matrix.loc[col_x, col_y] = np.nan
        else:
            # mutual_info_regression は (X, y) 形式なので reshape
            mi = mutual_info_regression(data[[col_x]], data[col_y], random_state=SEED)
            mi_matrix.loc[col_x, col_y] = mi[0]

# ヒートマップで可視化
plt.figure(figsize=(8, 6))
sns.heatmap(mi_matrix, cmap="coolwarm", annot=True, fmt=".2f")
plt.show()

ピアソン相関のヒートマップ図と似ている部分もありますが、異なる点として、緯度・経度と住宅価格にも他と比べてやや高めの依存関係が見られます。

これは、住宅価格と緯度経度には単純な線形の依存関係がありませんが、海沿い／内陸、北カリフォルニア／南カリフォルニアなどの地理的要因に非線形の依存関係があると推測されます。

### 目的変数の分布確認 (ヒストグラム)
目的変数であるMedHouseValの分布を、`seaborn`を使ってヒストグラムで可視化してみます。

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(data['MedHouseVal'], bins=50, kde=True)
plt.title('Median House Value Distribution')
plt.show()

気になる点として、最大値(5)に多くのサンプルが張り付いています(サチュレーションが起きている)。

これは、米国国勢調査のアンケートにおいて、50万ドル以上がすべて一つの項目にまとめられたためです。

## データ前処理
EDAでの結果をもとに、いくつかのデータ前処理を行ってみます。

### 欠損値の削除
EDAで確認した基本情報から、データ中に一定数の欠損値が存在することが分かります。

欠損値が明らかな不当データである場合、学習の精度に悪影響を与えることが多いです。

今回は、欠損値を含む行はすべて削除します。

データフレームなので、`dropna`メソッドで簡単に欠損値を含む行を削除できます。

※ `california housing dataset`には本来、欠損値はありません。今回は教育のため、あえて一部のデータを欠損値に置き換えてあります。

In [ ]:
data_dna = data.dropna()
data_dna

### 目的変数と説明変数の決定
今回は住宅価格の予想をしますので、`MedHouseVal`カラムを目的変数とします。

その他のカラムを説明変数として、データフレームを分割します。

In [ ]:
# 説明変数
X = data_dna.loc[:,"MedInc":"Longitude"]

# 目的変数
y = data_dna["MedHouseVal"]

print("==== 説明変数 ====")
print(X.head())

print("\n==== 目的変数 ====")
print(y.head())

なお一般的に、機械学習のコードにおいて説明変数を`X`(大文字)、目的変数を`y`(小文字)という変数に格納することが多いです。

今回も、そのような変数名にしています。

###  訓練データ、テストデータの分割
すべてのデータのうち、80%を訓練データに、20%をテストデータに割り当てます。

`scikit-learn`が提供する`train_test_split`関数で、データを分けることができます。

ここでは、`test_size`引数に0.2を指定することで、20%をテストデータに、残りを訓練データに分けています。

`X_train`および`y_train`が訓練データ、`X_test`および`y_test`がテストデータとなります。

In [ ]:
from sklearn.model_selection import train_test_split

# 説明変数のデータを訓練データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

print("Split completed.")

### 外れ値の除外
EDAで目的変数の分布を確認したとき、最大値(5)に多くのサンプルが張り付いていました。これは外れ値とみなすこともできます。

前処理として外れ値を除外するやり方もあり、特に線形の学習アルゴリズムでは効果があります。

しかし、現実ではこのようなサチュレーションしたデータはよく見られますし、アルゴリズムの選択によって学習への悪影響を減らすこともできますので、今回はこのままにします。

【注意事項】

もし外れ値の除去を行う場合、講義でご説明した通り、データリークを起こさないよう分割後のテストデータに対してのみ除去を行うという点に、くれぐれも注意してください。

## 特徴量エンジニアリング
学習のプロセスの中で、よりよい学習結果を得るために、新たな特徴量を作成したり、特徴量を削減したりすることもあります。例えば、以下のようなアプローチが考えられます。

- 「世帯ごとの部屋数」や「部屋あたり寝室比」などの特徴量を新たに作成
- PCAによる次元削減でデータを圧縮

今回はこれらの作業は行いませんが、実際に現場で機械学習を行う際は、このような観点も考えてみてください。

## 学習の実行
学習を行い、学習済みモデルを作成します。

アルゴリズムは、シンプルな線形回帰を使います。

`scikit-learn`では線形回帰のための`LinearRegression`クラスを提供しています。

なお`scikit-learn`では、機械学習モデルを実装したクラスを「**<font color="red">推定器(estimator)</font>**」と呼びます。

`LinearRegression`のインスタンスを作成し、訓練データを引数として`fit`メソッドを実行することで学習できます。

In [ ]:
from sklearn.linear_model import LinearRegression

# 学習の実行
model = LinearRegression()
model.fit(X_train, y_train)

print("Learning completed.")

## 係数の表示
ご参考までに、学習によって導出された、それぞれの説明変数の係数(重み)を確認してみましょう。

In [ ]:
# 係数の表示
coef_df = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": model.coef_
})
print("=== Coefficients ===")
print(coef_df)

ピアソン相関では相関の低かった、AveBedrmsやLatitude/Longitudeといった項目の係数の絶対値が大きくなっています。

ピアソン相関は、1対1の特徴量のみにしか着目しておらず、他の変数は一切考慮されません。

一方、線形回帰の回帰係数は、他の変数を固定したときの関係を示しており、解釈が異なります。

今回は特徴量同士に強い相関がみられるパターンもあったため、多重共線性の影響で、このような結果の差異が生じている可能性があります。

予測精度そのものにはあまり影響を及ぼさないことが多いですが、それぞれの説明変数が予測に与える影響を詳細にきちんと分析する場合などには、悪影響を及ぼします。

そのような観点を重視する場合は、機械学習モデルの選択や、特徴量エンジニアリングなどを再検討する必要があります。

## 予測の実行
学習済みモデルが作成できましたので、テストデータを用いて予測を行います。

テストデータを引数として、`LinearRegression`のインスタンスの`predict`メソッドを実行することで予測ができます。

In [ ]:
# 予測の実行
y_pred = model.predict(X_test)

print(f"Predict completed.")

## 学習済みモデルの評価
トレーニングの結果と、予測の結果を比較して学習済みモデルを評価します。

評価指標として、MSEとR2スコアを使用します。

これらの評価指標は、`scikit-learn`が提供する`mean_squared_error`関数および`r2_score`関数で簡単に計算できます。

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# 学習済みモデルの評価
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.3f}")
print(f"R2スコア: {r2:.3f}")

# 実際の価格と予測結果をプロット
fig = plt.figure()
ax = fig.subplots()
ax.scatter(y_test, y_pred, alpha=0.5)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color="red", linewidth=2)
ax.grid()
ax.set_title("True vs Predicted")
ax.set_xlabel("True Values")
ax.set_ylabel("Predicted Values")
fig.show()
# plt.close(fig) # 画像が消えてしまうため省略

MSEは値が小さいほど、R2スコアは値が1に近いほど、学習済みモデルが優れていると言えます。

可視化の結果を見ると、実際の価格が大きくなると、予測とのずれが大きくなっていくようです。

今回の結果は、いまひとつと言えそうです。

## アルゴリズムの変更
線形回帰の結果が今一つでしたので、アルゴリズムをランダムフォレストに変更してみます。

`scikit-learn`ではランダムフォレストによる回帰の推定器として、`RandomForestRegressor`を提供しています。

学習・予測まで行ったあと、MSEとR2スコアを線形回帰のものと比較してみます。

(こちらは実行にちょっと時間がかかります。)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 学習の実行
model_rf = RandomForestRegressor(random_state=SEED)
model_rf.fit(X_train, y_train)

# 予測の実行
y_pred_rf = model_rf.predict(X_test)

# モデルの評価 (線形回帰 vs ランダムフォレスト回帰)
print("MSE")
print(f"Linear Regression: {mean_squared_error(y_test, y_pred):.3f}")
print(f"    Random Forest: {mean_squared_error(y_test, y_pred_rf):.3f}")

print("\nR2")
print(f"Linear Regression: {r2_score(y_test, y_pred)}")
print(f"    Random Forest: {r2_score(y_test, y_pred_rf)}")

# 実際の価格と予測結果をプロット
print("\nPlot")
fig = plt.figure()
ax = fig.subplots()
ax.scatter(y_test, y_pred_rf, alpha=0.5)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color="red", linewidth=2)
ax.grid()
ax.set_title("California Housing: True vs Predicted")
ax.set_xlabel("True Values")
ax.set_ylabel("Predicted Values")
fig.show()
# plt.close(fig) # 画像が消えてしまうため省略

MSEはより小さく、R2スコアはより1に近づいたことから、結果が改善したことが分かります。

可視化結果を見ると、正解のラインに沿うように予測結果が分布していることから、結果が改善していることが分かります。

今回のデータはサチュレーションが起きていたり、他に非線形の相関も見られたため、非線形データにも比較的強いランダムフォレスト回帰で、精度が向上したと推測できます。

# 分類
次に、分類を実践します。

今回は、`breast cancer dataset`(乳がん検査のデータセット)を用いて、腫瘍の悪性・良性を分類します。

`breast cancer dataset`は、乳がんの検診で腫瘍が見つかった患者に対して腫瘍細胞を採取し、顕微鏡で分析した結果を数値化した情報です。次のような特徴量を持っています。

| カラム名 | 意味 | 説明 |
|-----------|----------------|------|
| mean radius | 平均半径 | 腫瘍細胞核の平均半径(中心から周までの距離) |
| mean texture | 平均テクスチャ | 画素輝度の標準偏差(画像の粒度の指標) |
| mean perimeter | 平均周囲長 | 腫瘍細胞核の平均周囲長 |
| mean area | 平均面積 | 腫瘍細胞核の平均面積 |
| mean smoothness | 平均平滑度 | 半径の長さの局所的変動(平滑さの指標) |
| mean compactness | 平均コンパクト度 | 周囲長² / 面積 − 1.0(形の密度の指標) |
| mean concavity | 平均凹度 | 輪郭のくぼみの強さの平均 |
| mean concave points | 平均凹点数 | 輪郭のくぼみの数の平均 |
| mean symmetry | 平均対称性 | 腫瘍形状の対称性の平均 |
| mean fractal dimension | 平均フラクタル次元 | “海岸線の近似度"のような不規則性の指標 |
| radius error | 半径の標準誤差 | 半径測定のばらつき |
| texture error | テクスチャの標準誤差 | テクスチャ測定のばらつき |
| perimeter error | 周囲長の標準誤差 | 周囲長測定のばらつき |
| area error | 面積の標準誤差 | 面積測定のばらつき |
| smoothness error | 平滑度の標準誤差 | 平滑度測定のばらつき |
| compactness error | コンパクト度の標準誤差 | コンパクト度測定のばらつき |
| concavity error | 凹度の標準誤差 | 凹度測定のばらつき |
| concave points error | 凹点数の標準誤差 | 凹点数測定のばらつき |
| symmetry error | 対称性の標準誤差 | 対称性測定のばらつき |
| fractal dimension error | フラクタル次元の標準誤差 | フラクタル次元測定のばらつき |
| worst radius | 最大半径 | 最大(最悪)半径値 |
| worst texture | 最大テクスチャ | 最大(最悪)テクスチャ値 |
| worst perimeter | 最大周囲長 | 最大(最悪)周囲長値 |
| worst area | 最大面積 | 最大(最悪)面積値 |
| worst smoothness | 最大平滑度 | 最大(最悪)平滑度 |
| worst compactness | 最大コンパクト度 | 最大(最悪)コンパクト度 |
| worst concavity | 最大凹度 | 最大(最悪)凹度 |
| worst concave points | 最大凹点数 | 最大(最悪)凹点数 |
| worst symmetry | 最大対称性 | 最大(最悪)対称性 |
| worst fractal dimension | 最大フラクタル次元 | 最大(最悪)フラクタル次元 |

分類のターゲットは"malignant"(悪性)と"benign"(良性)であり、0と1が割り当てられています。


## データの準備
`breast cancer dataset`は`scikit-learn`が標準提供していますので、そちらを使用します。

`load_breast_cancer`で読み込むことができます。

- 今回も`pandas`のデータフレームとして読み込むため、`as_frame`オプションにTrueを指定します。
- 目的変数(悪性/良性)のデータは`target`、説明変数のデータは`data`という名称で参照します。


In [ ]:
from sklearn.datasets import load_breast_cancer

# データセットの読み込み
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

# 読み込んだ説明変数データの先頭部を表示
print("Feature sample:")
print(X.head())

print("")

# 目的変数のラベル名を表示
print("Target names:", data.target_names)

"Target names"を見ると、"malignant"(悪性)と"benign"(良性)のラベルとなっています。

このどちらに当てはまるのかを分類します。

## 探索的データ分析 (EDA)
分類タスクでは、訓練データが不均衡ではないか(分類結果が偏っていないか)がとても重要です。

講義でご紹介したとおり、99%が悪性であれば、すべて「悪性」と答えるだけで正解率99%となってしまいます。

正解ラベルの個数を確認しましょう。

In [ ]:
print(data.target.value_counts())

偏りもなく、ぴったり半々でもないですね。

学習の効果が見えやすく、教育目的としてはとても「ちょうどよい」データといえます。

## 訓練データ・テストデータの準備
今回も、すべてのデータのうち、80%を訓練データに、20%をテストデータに割り当てます。

In [ ]:
# 説明変数のデータを訓練データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

print("Split completed.")

## データのスケーリング
今回は、アルゴリズムにサポートベクターマシン(SVM)を使います。

サポートベクターマシンにおいては、データをスケーリングすることが重要となります。

`scikit-learn`の`StandardScaler`モジュールを利用して、標準化を行うことができます。

In [ ]:
from sklearn.preprocessing import StandardScaler

# 訓練データ・テストデータの標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Standardize completed.")

講義でご説明した通り、スケーリングの手順を誤るとデータリークの恐れがあるのでご注意ください。

訓練データに対して行っている`fit_transform`メソッドは、統計情報の計算とスケーリングを行います。

テストデータに対して行っている`transform`は、スケーリングのみを行います。統計情報は、訓練データで計算したものを使います。

この手順であれば、データリークを起こさずにスケーリングを行うことができます。

## 学習・予測の実行

実際に学習を行い、学習済みモデルを作成します。

`scikit-learn`ではサポートベクターマシンによる分類の推定器として、`SVC`を提供しています。

回帰の場合と同様に、`fit`メソッドで学習を行い、`predict`メソッドで分類を行います。

In [ ]:
from sklearn.svm import SVC

# Create SVM model
svm_model = SVC(random_state=SEED)
svm_model.fit(X_train_scaled, y_train)

# Predict
y_pred = svm_model.predict(X_test_scaled)

print("fit and predict completed.")

## 学習済みモデルの評価
テストデータの正解ラベルと、分類の予測結果を比較して学習済みモデルを評価します。

評価指標として、`classification_report`関数と混同行列を使用します。

`classification_report`関数は、分類レポートとして適合率(precision)、再現率(recall)、F1スコア、正解率(accuracy)、マクロ平均、およびマイクロ平均を算出してくれます。

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

# 分類レポート
print("分類レポート:")
print(classification_report(y_test, y_pred, target_names=data.target_names))

# 混同行列
print("混同行列:")
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=data.target_names)
disp.plot(cmap='Blues')

今回は不均衡データではないので正解率(accuracy)に着目すると、かなり1に近い値となっているため、よい精度で分類できていることが分かります。

混合行列からも、不一致の個数が少ないことが分かります。

# クラスタリング
次に、クラスタリングを行ってみます。

ここでは`wine dataset`(ワイン品種のデータセット)を用いて、品種のクラスタリングをします。

`wine dataset`は、「アルコール度数/色の濃さ」などの13個の特徴量と、3種類のワインのいずれかを示すラベルで構成される表形式データセットです。次のような特徴量を持っています。

| カラム名 | 意味日本語 | 説明 |
|-----------|----------------|------|
| alcohol | アルコール度数 | ワイン中のアルコール含有量 |
| malic_acid | リンゴ酸 | 酸味を与える有機酸の一種(リンゴ酸)の含有量 |
| ash | 灰分 | 灰分(ミネラル分)の含有量 |
| alcalinity_of_ash | 灰分のアルカリ度 | 灰分のアルカリ性の強さを示す指標 |
| magnesium | マグネシウム含有量 | ワイン中のマグネシウムの含有量(mg/L) |
| total_phenols | 総フェノール量 | 抗酸化物質であるフェノール化合物の総量 |
| flavanoids | フラボノイド含有量 | フェノール化合物の一種(フラボノイド)の含有量 |
| nonflavanoid_phenols | 非フラボノイドフェノール量 | フラボノイド以外のフェノール化合物の量 |
| proanthocyanins | プロアントシアニジン量 | タンニンの一種(苦味・渋味の要素) |
| color_intensity | 色の濃さ | ワインの色の濃度(視覚的強度) |
| hue | 色相 | ワインの色合い(赤みや黄みの度合い) |
| od280/od315_of_diluted_wines | OD280/OD315比 | 希釈ワインの280nmと315nmでの吸光度比(タンパク質含量と熟成の指標) |
| proline | プロリン含有量 | アミノ酸の一種で、ワインの熟成度・品種特性を反映する成分 |

正解ラベルとして、ワインの品種を示すクラス1～3のデータを持っており、整数の1, 2, 3が割り当てられています。

【注意事項】

`wine dataset`には正解ラベル(ワインの品種が何か)が含まれていますが、クラスタリングは「教師なし学習」なので、正解ラベルを除いたデータを使って学習します。

このようにすることで、あくまで教育目的として、クラスタリングが適切に行われたのかの「答え合わせ」ができます。

## データの準備
`wine dataset`も、`scikit-learn`が標準提供しています。

`load_wine`で読み込むことができます。

- 今回も`pandas`のデータフレームとして読み込むため、`as_frame`オプションにTrueを指定します。
- 品種のデータは`target`で参照できますが、学習には使用しません。答え合わせでのみ使用します。

In [ ]:
from sklearn.datasets import load_wine

# データセットの読み込み
wine = load_wine(as_frame=True)
X = wine.data
y = wine.target # 答え合わせで使用する

print(X.head())
print("")

# 目的変数のラベル名を表示 (品種数を確認)
print("Target names:", wine.target_names)

## データのスケーリング
今回は、アルゴリズムにk-meansを使います。

k-meansにおいても、データをスケーリングすることが重要となります。

ここでも`scikit-learn`の`StandardScaler`モジュールを利用して、標準化を行います。

教師なし学習なので、Xに対してのみ標準化を実行します。

In [ ]:
# データの標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Standardize completed.")

## クラスタリングの実行
実際に学習を行い、学習済みモデルを作成します。

`scikit-learn`ではk-meansによるクラスタリングの推定器として、`KMeans`を提供しています。

`KMeans`のインスタンスを作成し、`fit_predict`メソッドを実行することでクラスタリングを行います。

`n_clusters`はクラスタの数を指定します。`wine dataset`は3つの品種なので、3を指定します。

In [ ]:
from sklearn.cluster import KMeans

# クラスタリングの実行
kmeans = KMeans(n_clusters=3, random_state=SEED)
labels = kmeans.fit_predict(X_scaled)

print("Clustering completed.")

## 学習済みモデルの評価
クラスタリング結果の妥当性について、学習済みモデルを評価します。

### ARI、シルエットスコア
まずは評価指標として、ARIとシルエットスコアを使用します。

今回は正解データが存在するため、ARIを使うことができます。

実際のクラスタリングでは基本的に正解データがありませんので、シルエットスコアなどのクラスタの特性を示す評価指標や、可視化による評価のみ行うことになります。

In [ ]:
from sklearn.metrics import adjusted_rand_score, silhouette_score

ari = adjusted_rand_score(y, labels)
sil = silhouette_score(X_scaled, labels)
print(f"ARI: {ari:.3f}")
print(f"シルエットスコア: {sil:.3f}")

ARIは+1が完全一致であり、比較的高い精度で分類ができていると言えます。

シルエットスコアは+1に近いほどクラスタ間の境界がはっきりしており、0に近いほど境界があいまいです。0の方に近いため、境界が完全に独立ではないようです。

### クロス集計
正解データと、クラスタリングしたクラスタラベルの関係を集計します。

クロス集計も、あくまで正解データが存在する場合のみ使用できますので、注意してください。

`pandas`の`crosstab`関数を使用して表にしてみます。

なお、k-meansは教師なし学習なので、正解ラベルの番号(0, 1, 2)と、クラスタラベルの番号(0, 1, 2)は必ずしも一致しません。「同じ番号＝同じクラス」とは限らない点に注意してください。

In [ ]:
# クラスタラベルと真のラベルをクロス集計
conf_mat = pd.crosstab(y, labels, rownames=['True Label'], colnames=['Cluster'])
conf_mat

ARIやシルエットスコアを裏付けるように、比較的高い精度で分類ができているものの、一部境界があいまいできれいに分類されていないデータもあります。

## 未知データへの凡化性能について
回帰や分類と異なり、クラスタリングではデータを訓練データとテストデータに分割し、未知データ(テストデータ)でも適切に動作するかを評価しませんでした。

クラスタリングは、既存データの構造やパターンを分析するのが目的であることが非常に多いためです。

もし新しいデータを自動でクラスタリングしたいというユースケースであれば、回帰や分類のようなテストデータでの検証は有効です。

ただしクラスタリングのような教師なし学習では「正解」がないため、精度という観点ではなく、シルエットスコアなどクラスタの特性を示す評価指標に大きなぶれが生じないか、などの観点がメインとなります。

# 次元削減
最後に、クラスタリングの続きとして、次元削減を実践します。

`wine dataset`は次元数(説明変数の数)が13です。

計算リソースの都合でもっと説明変数を圧縮したい場合や、散布図などで可視化して傾向を確認したい場合などは、次元削減が有効です。

## 説明変数の圧縮
まずは、次元削減により説明変数の数を圧縮してみます。

アルゴリズムは主成分分析(PCA)を使います。

`scikit-learn`では主成分分析による次元削減の推定器として、`PCA`を提供しています。

`PCA`のインスタンスを作成し、`fit_transform`メソッドを実行することで次元削減を行います。

`n_components`は削減後の次元数を指定します。

次元削減をした結果、削減前の情報をどの程度維持しているかの目安として、累積寄与率を確認します。

累積寄与率が0.8以上あれば、削減前の情報をかなり維持できているというひとつの目安になります。

ここでは、次元数を2～6に削減したそれぞれの累積寄与率を確認してみます。

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

for n in range(2, 7):
    # PCAでn次元に圧縮
    pca = PCA(n_components=n, random_state=SEED)
    pca.fit_transform(X_scaled)

    # 累積寄与率
    print(f"n={n}: Cumulative explained variance: {np.sum(pca.explained_variance_ratio_)}")

次元数を5まで削減しても、累積寄与率0.8以上を維持できています。

次元数5で、もう一度クラスタリングしてみましょう。

In [ ]:
# PCAで5次元に圧縮
pca5 = PCA(n_components=5, random_state=SEED)
X_pca5 = pca.fit_transform(X_scaled)

# 次元数5でクラスタリング
kmeans_pca5 = KMeans(n_clusters=3, random_state=SEED)
labels_pca5 = kmeans_pca5.fit_predict(X_pca5)

# 学習済みモデルの評価
ari_pca5 = adjusted_rand_score(y, labels_pca5)
sil_pca5 = silhouette_score(X_pca5, labels_pca5)
print(f"ARI: {ari_pca5:.3f}")
print(f"シルエットスコア: {sil_pca5:.3f}")

# クラスタラベルと真のラベルをクロス集計
conf_mat = pd.crosstab(y, labels_pca5, rownames=['True Label'], colnames=['Cluster'])
conf_mat

ARIは次元削減前と同一水準です。

シルエットスコアは少し大きくなりました。次元削減によりノイズが除去され、境界が明確になった可能性があります。

クロス集計の結果は同一であることから、クラスタリングの精度に影響を与えていないことも分かります。

次元削減により、説明変数の数を13から5へ、半分以下に削減することができました。

## 散布図による可視化
先ほどの5次元でのクラスタリング結果について、正解ラベルと、クラスタリングしたクラスタラベルを比較する散布図を作成します。

5次元データをそのまま散布図にはできないので、2次元まで削減したデータに5次元データを射影して散布図にします。

<img src="https://github.com/tendo-sms/python_intermediate/raw/main/2025/figure/pca5to2.png" width="80%" alt="パレートフロントのイメージ">

In [ ]:
# PCAで2次元に圧縮
pca2 = PCA(n_components=2, random_state=SEED)
X_pca2 = pca2.fit_transform(X_scaled)

# 累積寄与率
print(f"Cumulative explained variance: {np.sum(pca2.explained_variance_ratio_)}")

なお、2次元に圧縮すると累積寄与率は55%程度まで下がりました。

これをクラスタリングに使用するのは実用に耐えませんが、あくまで傾向を掴むための可視化が目的であれば、ある程度有効です。

それでは、散布図を描画してみます。

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,5))

# 真のラベル分布
axes[0].scatter(X_pca2[:,0], X_pca2[:,1], c=y, cmap='viridis', alpha=0.6)
axes[0].set_title("True Labels (Wine Target)")
axes[0].set_xlabel("PC1")
axes[0].set_ylabel("PC2")

# KMeansクラスタ分布
axes[1].scatter(X_pca2[:,0], X_pca2[:,1], c=labels_pca5, cmap='viridis', alpha=0.6)
axes[1].scatter(kmeans_pca5.cluster_centers_[:,0],
                kmeans_pca5.cluster_centers_[:,1],
                c='red', marker='X', s=200, edgecolor='black', label='Centroids')
axes[1].set_title("K-Means Cluster Labels")
axes[1].set_xlabel("PC1")
axes[1].set_ylabel("PC2")
axes[1].legend()

plt.tight_layout()
plt.show()

これらの可視化の結果からみても、境界の重なりで若干の外れはありますが、ほぼ適切にクラスタリングできていることが分かります。

# おわりに
以上で、「機械学習 ハンズオン」の講義は終了となります。

今回の内容を参考に、scikit-learnのリファレンスを見ていろいろなデータ・アルゴリズムを試してみてください。

このあとは、グループワークにチャレンジしていただきます。